# VC Verification

This notebook walks through verifying a Data Integrity proof on a secured document according to the TBD schnorr-secp256k1-jcs-2024 cryptosuite

# Verify Proof (schnorr-secp256k1-jcs-2024 cryptosuite)
The following algorithm specifies how to verify a data integrity proof given an secured data document. Required inputs are an secured data document (map securedDocument). This algorithm returns a verification result, which is a struct whose items are:

- verified (true or false)
- verifiedDocument (if verified is true, an unsecured data document; otherwise Null)

1. Let unsecuredDocument be a copy of securedDocument with the proof value removed.
2. Let proofOptions be the result of a copy of securedDocument.proof with proofValue removed.
3. Let proofBytes be the Multibase decoded base58-btc value in securedDocument.proof.proofValue.
4. If proofOptions.@context exists:
    - Check that the securedDocument.@context starts with all values contained in the proofOptions.@context in the same order. Otherwise, set verified to false and skip to the last step.
    - Set unsecuredDocument.@context equal to proofOptions.@context.
5. Let transformedData be the result of running the algorithm in Section XXX Transformation (schnorr-secp256k1-jcs-2024) with unsecuredDocument and proofOptions passed as parameters.
6. Let proofConfig be the result of running the algorithm in Section XXX Proof Configuration (schnorr-secp256k1-jcs-2024) with proofOptions passed as the parameter.
7. Let hashData be the result of running the algorithm in Section XXX Hashing (schnorr-secp256k1-jcs-2024) with transformedData and proofConfig passed as a parameters.
8. Let verified be the result of running the algorithm in Section XXX Proof Verification (schnorr-secp256k1-jcs-2024) on hashData, proofBytes, and proofConfig.
9. Return a verification result with items:
- verified
- verifiedDocument (if verified is true, unsecuredDocument; otherwise, Null)

## 0.1. Add libbtc1 python library to path

In [37]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

## 0.2 Get secured document input

**You can create your own secured VC and copy it across in the VC Signing notebook**

In [2]:
secured_vc = {'@context': ['https://www.w3.org/ns/credentials/v2', 'https://www.w3.org/ns/credentials/examples/v2'], 'id': 'http://university.example/credentials/58473', 'type': ['VerifiableCredential', 'ExampleAlumniCredential'], 'validFrom': '2020-01-01T00:00:00Z', 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21', 'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1', 'name': 'Example University'}}, 'issuer': 'did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65', 'proof': {'type': 'DataIntegrityProof', 'cryptosuite': 'schnorr-secp256k1-jcs-2024', 'verificationMethod': 'did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey', 'proofPurpose': 'assertionMethod', '@context': ['https://www.w3.org/ns/credentials/v2', 'https://www.w3.org/ns/credentials/examples/v2'], 'proofValue': 'zsxM9je5iKynKyN6rRPi9QjTWpG6inJ1umwGfnCo4fiu4MqYf46PLd4TE2wVZvdZegDuC6xL6n3Kj8S1PbC8tmTm'}}

In [3]:
import json
print(json.dumps(secured_vc, indent=2))

{
  "@context": [
    "https://www.w3.org/ns/credentials/v2",
    "https://www.w3.org/ns/credentials/examples/v2"
  ],
  "id": "http://university.example/credentials/58473",
  "type": [
    "VerifiableCredential",
    "ExampleAlumniCredential"
  ],
  "validFrom": "2020-01-01T00:00:00Z",
  "credentialSubject": {
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": "Example University"
    }
  },
  "issuer": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
  "proof": {
    "type": "DataIntegrityProof",
    "cryptosuite": "schnorr-secp256k1-jcs-2024",
    "verificationMethod": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey",
    "proofPurpose": "assertionMethod",
    "@context": [
      "https://www.w3.org/ns/credentials/v2",
      "https://www.w3.org/ns/credentials/examples/v2"
    ],
    "proofValue": "zsxM9je5iKynKyN6rRPi9QjTWpG6inJ1umwGfnC

## 1. Let unsecuredDocument be a copy of securedDocument with the proof value removed.

In [4]:
import copy
unsecured_document = copy.deepcopy(secured_vc)

In [5]:
del unsecured_document["proof"]

In [6]:
print("Unsecured Document : ", json.dumps(unsecured_document, indent=2))

Unsecured Document :  {
  "@context": [
    "https://www.w3.org/ns/credentials/v2",
    "https://www.w3.org/ns/credentials/examples/v2"
  ],
  "id": "http://university.example/credentials/58473",
  "type": [
    "VerifiableCredential",
    "ExampleAlumniCredential"
  ],
  "validFrom": "2020-01-01T00:00:00Z",
  "credentialSubject": {
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": "Example University"
    }
  },
  "issuer": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65"
}


## 2. Let proofOptions be the result of a copy of securedDocument.proof with proofValue removed.


In [7]:
proof_options = copy.deepcopy(secured_vc["proof"])
del proof_options["proofValue"]

print("Proof Options : \n", json.dumps(proof_options,indent=2))

Proof Options : 
 {
  "type": "DataIntegrityProof",
  "cryptosuite": "schnorr-secp256k1-jcs-2024",
  "verificationMethod": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey",
  "proofPurpose": "assertionMethod",
  "@context": [
    "https://www.w3.org/ns/credentials/v2",
    "https://www.w3.org/ns/credentials/examples/v2"
  ]
}


## 3. Let proofBytes be the Multibase decoded base58-btc value in securedDocument.proof.proofValue.


In [8]:
from multiformats import multibase
proof_value = secured_vc["proof"]["proofValue"]
proof_bytes = multibase.decode(proof_value)

print("Proof Bytes : ", proof_bytes)

Proof Bytes :  b"+\xf0\xe6\xff\xe7\x95\xee\xbbb\x14\xf8\r\xa9\x88\x81\x82\x02@[\xb3\x98#j\xba\x7f\xf0Z\x9f&\x12\x01\xa3L\xd2\xc6\x92 L\x19G\x8d\xfb\xba\xc5\xa7\xfd\x8b\xbe\x16'n\xb9\t\xa1\xad,f+R\x1c\x96X\xf9\x08"


## 4. If proofOptions.@context exists:
    - Check that the securedDocument.@context starts with all values contained in the proofOptions.@context in the same order. Otherwise, set verified to false and skip to the last step.
    - Set unsecuredDocument.@context equal to proofOptions.@context.

In [9]:
if  "@context" in proof_options:
    for i in range(len(proof_options["@context"])):
        assert proof_options["@context"][i] == secured_vc["@context"][i]
    unsecured_document["@context"] = proof_options["@context"]
else: 
    print("No @context in proofOptions")

## 5. Let transformedData be the result of running the algorithm in Section XXX Transformation (schnorr-secp256k1-jcs-2024) with unsecuredDocument and proofOptions passed as parameters.


### Spec Text

The following algorithm specifies how to transform an unsecured input document into a transformed document that is ready to be provided as input to the hashing algorithm in Section XXX Hashing (schnorr-secp256k1-jcs-2024).

Required inputs to this algorithm are an unsecured data document (unsecuredDocument) and transformation options (options). The transformation options MUST contain a type identifier for the cryptographic suite (type) and a cryptosuite identifier (cryptosuite). A transformed data document is produced as output. Whenever this algorithm encodes strings, it MUST use UTF-8 encoding.

1. If options.type is not set to the string DataIntegrityProof and options.cryptosuite is not set to the string schnorr-secp256k1-jcs-2024, an error MUST be raised that SHOULD convey an error type of PROOF_VERIFICATION_ERROR.
2. Let canonicalDocument be the result of applying the JSON Canonicalization Scheme [RFC8785] to a JSON serialization of the unsecuredDocument.
3. Return canonicalDocument as the transformed data document.

In [10]:
## This would be passed into the algorithm.
options = proof_options

assert options["type"] == "DataIntegrityProof"
assert options["cryptosuite"] == "schnorr-secp256k1-jcs-2024"

In [11]:
import jcs
canonical_document = jcs.canonicalize(unsecured_document)

In [12]:
print("Returning canonical document : ", canonical_document)
transformed_document= canonical_document

Returning canonical document :  b'{"@context":["https://www.w3.org/ns/credentials/v2","https://www.w3.org/ns/credentials/examples/v2"],"credentialSubject":{"alumniOf":{"id":"did:example:c276e12ec21ebfeb1f712ebc6f1","name":"Example University"},"id":"did:example:ebfeb1f712ebc6f1c276e12ec21"},"id":"http://university.example/credentials/58473","issuer":"did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65","type":["VerifiableCredential","ExampleAlumniCredential"],"validFrom":"2020-01-01T00:00:00Z"}'


## 6. Let proofConfig be the result of running the algorithm in Section XXX Proof Configuration (schnorr-secp256k1-jcs-2024) with proofOptions passed as the parameter.


The following algorithm specifies how to generate a proof configuration from a set of proof options that is used as input to the proof hashing algorithm.

The required inputs to this algorithm are proof options (options). The proof options MUST contain a type identifier for the cryptographic suite (type) and MUST contain a cryptosuite identifier (cryptosuite). A proof configuration object is produced as output.

1. Let proofConfig be a clone of the options object.
2. If proofConfig.type is not set to DataIntegrityProof or proofConfig.cryptosuite is not set to schnorr-secp256k1-jcs-2024, an error MUST be raised that SHOULD convey an error type of PROOF_GENERATION_ERROR.
3. If proofConfig.created is set to a value that is not a valid [XMLSCHEMA11-2] datetime, an error MUST be raised and SHOULD convey an error type of PROOF_GENERATION_ERROR.
4. Let canonicalProofConfig be the result of applying the JSON Canonicalization Scheme [RFC8785] to the proofConfig.
Return canonicalProofConfig.

In [13]:
# Passed in 
options = copy.deepcopy(proof_options)

### 6.1 Let proofConfig be a clone of the options object.

In [14]:
proofConfig = copy.deepcopy(options)

### 6.2. If proofConfig.type is not set to DataIntegrityProof or proofConfig.cryptosuite is not set to schnorr-secp256k1-jcs-2024, an error MUST be raised that SHOULD convey an error type of PROOF_GENERATION_ERROR.

In [15]:
assert proofConfig["type"] == "DataIntegrityProof"
assert proofConfig["cryptosuite"] == "schnorr-secp256k1-jcs-2024"

### 6.3. If proofConfig.created is set to a value that is not a valid [XMLSCHEMA11-2] datetime, an error MUST be raised and SHOULD convey an error type of PROOF_GENERATION_ERROR.


In [16]:
print("TODO: Check datetime if created set")

TODO: Check datetime if created set


### 6.4. Let canonicalProofConfig be the result of applying the JSON Canonicalization Scheme [RFC8785] to the proofConfig.


In [17]:
import jcs
canonicalProofConfig = jcs.canonicalize(proofConfig)

### 6.5 Return canonicalProofConfig

In [18]:
proof_config = canonicalProofConfig
print("Return canonicalProofConfig : ", canonicalProofConfig)

Return canonicalProofConfig :  b'{"@context":["https://www.w3.org/ns/credentials/v2","https://www.w3.org/ns/credentials/examples/v2"],"cryptosuite":"schnorr-secp256k1-jcs-2024","proofPurpose":"assertionMethod","type":"DataIntegrityProof","verificationMethod":"did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey"}'


## 7. Let hashData be the result of running the algorithm in Section XXX Hashing (schnorr-secp256k1-jcs-2024) with transformedData and proofConfig passed as a parameters.


In [19]:
from buidl.helper import sha256
data_to_hash = proof_config + transformed_document
hash_data = sha256(data_to_hash)
print("Return hashData", hash_data)

Return hashData b'\xc9X<\xf9p\xb1C\xba\x84\x18 \x0fr$<U\xe9\\R\x13\xc1\xb1\xcf\x04\xe0L\xbf\x01\x06\xa9\xa4\t'


## 8. Let verified be the result of running the algorithm in Section XXX Proof Verification (schnorr-secp256k1-jcs-2024) on hashData, proofBytes, and proofConfig.


### Spec Text

The following algorithm specifies how to verify a digital signature from a set of cryptographic hash data. This algorithm is designed to be used in conjunction with the algorithms defined in the Data Integrity [VC-DATA-INTEGRITY] specification, Section 4: Algorithms. Required inputs are cryptographic hash data (hashData), a digital signature (proofBytes) and proof options (options). A verification result represented as a boolean value is produced as output.

1. Let publicKeyBytes be the result of retrieving the public key bytes associated with the options.verificationMethod value as described in the Data Integrity [VC-DATA-INTEGRITY] specification, Section 4: Retrieving Cryptographic Material.
2. Let verificationResult be the result of applying the verification algorithm for the Schnorr Alogrithm [BIP340], using the Secp256k1 variant, with hashData as the data to be verified against the proofBytes using the public key specified by publicKeyBytes.
3. Return verificationResult as the verification result.


### 8.1 Let publicKeyBytes be the result of retrieving the public key bytes associated with the options.verificationMethod value as described in the Data Integrity [VC-DATA-INTEGRITY] specification, Section 4: Retrieving Cryptographic Material.

Need to dereference the verificationMethod. 

In [20]:
verification_method_id = options["verificationMethod"]
verification_method_id




'did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey'

In [21]:
from urllib import parse
did_url = parse.urlparse(verification_method_id)

did_btc1 = f"did:{did_url.path}"
fragment = did_url.fragment

In [22]:
from libbtc1.did import resolve

did_document = resolve(did_btc1)

verification_method = None
for vm in did_document["verificationMethod"]:
    vm_id = vm["id"]
    if vm_id[0] == '#':
        print("Relative URL")
        vm_id = did_btc1 + vm_id
    if vm_id == verification_method_id:
        print("Verification Method Found")
        verification_method = vm
        break

if verification_method == None:
    raise Exception("No verification method found")
else:
    print(json.dumps(verification_method, indent=2))

Relative URL
Verification Method Found
{
  "id": "#initialKey",
  "type": "Multikey",
  "controller": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
  "publicKeyMultibase": "z66PwJnYvwJLhGrVc8vcuUkKs99sKCzYRM2HQ2gDCGTAStHk"
}


In [29]:
from libbtc1.verificationMethod import get_key_for_verification_method

public_key = get_key_for_verification_method(verification_method)

print("Public key bytes : ", public_key.sec())

Public key bytes :  b"\x02\x9a\xd5\xf6\xa8]'\xeei\xb13\xae\xd2s\xb4\xf2\xf5\xd7\x0e\xd4\xa7\x16u\x01\x9c\x1av\xf0Lf5&\xef"


### 8.2. Let verificationResult be the result of applying the verification algorithm for the Schnorr Alogrithm [BIP340], using the Secp256k1 variant, with hashData as the data to be verified against the proofBytes using the public key specified by publicKeyBytes.

In [32]:
from buidl.ecc import SchnorrSignature

sig = SchnorrSignature.parse(proof_bytes)
verification_result = public_key.verify_schnorr(hash_data,sig)

In [33]:
verification_result

True

### 8.3. Return verificationResult as the verification result.

In [34]:
print("Return verificationResult : ", verification_result)

Return verificationResult :  True


## 9. Return a verification result with items:
- verified
- verifiedDocument (if verified is true, unsecuredDocument; otherwise, Null)

In [35]:
proof_verification_result = {}

proof_verification_result["verified"] = verification_result
if verification_result:
    proof_verification_result["verifiedDocument"] = unsecured_document
else:
    proof_verification_result["verifiedDocument"] = None



In [36]:
print("Proof Verification Result \n", json.dumps(proof_verification_result,indent=2))

Proof Verification Result 
 {
  "verified": true,
  "verifiedDocument": {
    "@context": [
      "https://www.w3.org/ns/credentials/v2",
      "https://www.w3.org/ns/credentials/examples/v2"
    ],
    "id": "http://university.example/credentials/58473",
    "type": [
      "VerifiableCredential",
      "ExampleAlumniCredential"
    ],
    "validFrom": "2020-01-01T00:00:00Z",
    "credentialSubject": {
      "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
      "alumniOf": {
        "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
        "name": "Example University"
      }
    },
    "issuer": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65"
  }
}
